# Create SMC claim dataset tables
This notebook guides through the creation of tables from the SQLite database. Flat data tables are needed for downstream tasks, e.g., annotation and model training. 

**Table of content:**
1. Create metadata table
2. Create data table
3. Create slices to be labeled

In [1]:
import sqlite3
import os

import pandas as pd

from config import DB_PATH, BASE_DIR

### 1. Create metadata table

In [2]:
connection = sqlite3.connect(DB_PATH)

In [5]:
metadata_table = pd.read_sql_query("SELECT pb_ID, title, date FROM Press_Briefing", connection)  # + topic , resort
metadata_table.head()

,pb_ID,title,date
0,1,Welche nationale Teststrategie bringt uns opti...,09.10.2020
1,2,"Früherkennung von Alzheimer – Je früher, desto...",15.09.2017
2,3,Desinformation vor der Europawahl – Wie groß i...,12.04.2019
3,4,Wie lassen sich der Kampf gegen den Klimawande...,08.06.2021
4,5,Zukünftige Teststrategien auf SARS-CoV-2 und ...,09.04.2020


### 2. Create data table

In [19]:
data_table = pd.read_sql_query("""SELECT Sentence.sentence_ID, title, sentence
        FROM Sentence 
        JOIN Press_Briefing ON Sentence.pb_ID = Press_Briefing.pb_ID""", connection)
data_table.head()

,sentence_ID,title,sentence
0,1,Welche nationale Teststrategie bringt uns opti...,Transkript
1,2,Welche nationale Teststrategie bringt uns opti...,Herzlich willkommen zum heutigen Press Briefin...
2,3,Welche nationale Teststrategie bringt uns opti...,"Mein Name ist Moderatorin, ich bin Redakteurin..."
3,4,Welche nationale Teststrategie bringt uns opti...,Ich begrüße die teilnehmenden Journalisten und...
4,5,Welche nationale Teststrategie bringt uns opti...,"Der Herbst ist da, die Infektionszahlen steige..."


In [22]:
data_table.to_csv(os.path.join(BASE_DIR, "SMC_claim_sentences.csv"), index=False)

### 3. Create slices to be labeled
Sentences with over 99% claim probability:

In [2]:
data = pd.read_csv(os.path.join("data", "SMC_dataset", "pre_labeled", "SMC_claim_setences_pre.csv"), names=["sentence_ID", "title", "sentence", "prob"])
data["label"] = None  # Column for the label
data.head()

,sentence_ID,title,sentence,prob,label
0,1,Welche nationale Teststrategie bringt uns opti...,Transkript,0.001865,None
1,2,Welche nationale Teststrategie bringt uns opti...,Herzlich willkommen zum heutigen Press Briefin...,0.002239,None
2,3,Welche nationale Teststrategie bringt uns opti...,"Mein Name ist Moderatorin, ich bin Redakteurin...",0.002861,None
3,4,Welche nationale Teststrategie bringt uns opti...,Ich begrüße die teilnehmenden Journalisten und...,0.004546,None
4,5,Welche nationale Teststrategie bringt uns opti...,"Der Herbst ist da, die Infektionszahlen steige...",0.003052,None


In [3]:
dataset_99 = data[data["prob"]>= 0.99]
dataset_99.head()

,sentence_ID,title,sentence,prob,label
19,20,Welche nationale Teststrategie bringt uns opti...,Bitte wählen Sie die Option an alle Teilnehmer...,0.992950,None
25,26,Welche nationale Teststrategie bringt uns opti...,"Aber ich möchte nochmal betonen, dass aus mein...",0.992971,None
30,31,Welche nationale Teststrategie bringt uns opti...,"Wenn dieser Test positiv ausfällt, ist aber au...",0.993059,None
34,35,Welche nationale Teststrategie bringt uns opti...,"Genau das zweite, die Sensitivität ist nicht s...",0.992652,None
42,43,Welche nationale Teststrategie bringt uns opti...,Diejenigen werden aber unter Umständen auch ni...,0.993070,None


In [4]:
dataset_99.to_json(os.path.join("data", "SMC_dataset", "pre_labeled", "SMC_claim_sentences_claims_99.jsonl"), orient='records', lines=True)  # aka v2.jsonl

Sentences with claim probability between 20% and 80%:

In [10]:
dataset_20_80 = data[data["prob"].between(0.20, 0.80, inclusive=True)]
dataset_20_80.head()

/tmp/ipykernel_2706/1292459789.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  dataset_20_80 = data[data["prob"].between(0.20, 0.80, inclusive=True)]


,sentence_ID,title,sentence,prob,label
14,15,Welche nationale Teststrategie bringt uns opti...,Allerdings diskutieren auch weitere Bereiche z...,0.353402,None
21,22,Welche nationale Teststrategie bringt uns opti...,Sie ist Direktorin des Instituts für medizinis...,0.366258,None
22,23,Welche nationale Teststrategie bringt uns opti...,"Hallo, Frau Ciesek.",0.285915,None
35,36,Welche nationale Teststrategie bringt uns opti...,Die Viruslast ist am höchsten kurz vor Auftrit...,0.262825,None
44,45,Welche nationale Teststrategie bringt uns opti...,Natürlich würde die PCR es ein bisschen früher...,0.238853,None


In [11]:
dataset_20_80.to_json(os.path.join("data", "SMC_dataset", "pre_labeled", "SMC_claim_sentences_claims_20_80.jsonl"), orient='records', lines=True)  # aka v2.jsonl